
# Workshop Tutorial: Leveraging AMD GPUs with DeepSeek-R1

Welcome to this hands-on workshop! Throughout this tutorial, we'll leverage AMD GPUs to deploy powerful language models like DeepSeek-R1. We'll cover setting up an AI development environment and explore three main practical applications:

1. **Advanced Chatbot:** Using Open WebUI to create a sophisticated chatbot with web search and file interaction capabilities.
2. **Code Development Assistant:** Installing and utilizing the CodeGPT VS Code extension to perform code analysis and pair programming tasks.
3. **Web-Based AI Agent:** Deploying a web agent capable of autonomous web browsing connected to our custom AI endpoint.

Let's dive in!

## Table of Contents

- [Step 1: Access AMD GPUs](#step1)
- [Step 2: Docker Installation and Configuration](#step2)
- [Step 3: Launching vLLM Server on AMD GPUs](#step3)
- [Step 4: Advanced Chatbot with Open WebUI](#step4)
- [Step 5: Chatbot Testing with DeepSeek-R1](#step5)
- [Step 6: Code Development Assistant Using CodeGPT](#step6)
- [Step 7: Build a Snake Game with CodeGPT](#step7)
- [Step 8: Optional Advanced Challenge - Pac-Man](#step8)
- [Step 9: Deploying Your Web-Based AI Agent](#step9)

<a id="step1"></a>

## Step 1: Access AMD GPUs

- **For Developers:** Request access via AMD’s Developer Cloud application [here](https://www.amd.com/en/forms/registration/developer-cloud-application.html).
- **Enterprise Users:** Check out AMD’s cloud partners [here](https://www.amd.com/en/developer/resources/rocm-hub/dev-ai.html).

<a id="step2"></a>

## Step 2: Docker Installation and Configuration
Ensure Docker is installed and properly configured on your system. Follow [Docker's official installation guide](https://docs.docker.com/get-docker/) if necessary.

**Docker Permissions:**

Allow non-root Docker access by running:

```bash
sudo usermod -aG docker $USER
newgrp docker
```

Verify Docker installation:

```bash
docker run hello-world
```

<a id="step3"></a>

## Step 3: Launching vLLM Server on AMD GPUs
### Start vLLM Docker Container
Launch an interactive vLLM Docker container:

```bash
docker run -it --rm \
  --network=host \
  --device=/dev/kfd \
  --device=/dev/dri \
  --group-add=video \
  --ipc=host \
  --cap-add=SYS_PTRACE \
  --security-opt seccomp=unconfined \
  --shm-size 16G \
  rocm/vllm-dev:main
```

### Deploy DeepSeek-R1 Model with vLLM
Run the following commands within your Docker container to start the vLLM server:

```bash
export VLLM_SEED=42
export VLLM_MLA_DISABLE=0
export VLLM_USE_TRITON_FLASH_ATTN=0
export VLLM_USE_ROCM_FP8_FLASH_ATTN=0
export VLLM_FP8_PADDING=1
export VLLM_USE_AITER_MOE=1
export VLLM_USE_AITER_BLOCK_GEMM=1
export VLLM_USE_AITER_MLA=0

vllm serve deepseek-ai/DeepSeek-R1-Distill-Qwen-32B \
    --host 0.0.0.0 \
    --port PORT_NUMBER \
    --api-key abc-123 \
    --trust-remote-code \
    --seed 42
```

**Note:** Select an appropriate port number that is open to the public. Ensure network and firewall settings allow connections to your server via selected port number.

Upon successful launch, your server should be accepting incoming traffic through an OpenAI-compatible API. Retrieve the public IP address of your server (typically available from your cloud provider's dashboard or by running `curl ifconfig.me` within the container or server shell). Verify the endpoint's reachability by executing the following from your local laptop:

```
curl http://YOUR_SERVER_PUBLIC_IP:PORT_NUMBER/v1/models -H "Authorization: Bearer abc-123"
```

<a id="step4"></a>

## Step 4: Advanced Chatbot with Open WebUI
Follow the installation instructions from the [Open WebUI GitHub repository](https://github.com/open-webui/open-webui).

After installation, configure your endpoint URL in the Open WebUI client as follows:

- Navigate to `Settings` as shown in the image below:

![OpenWebUI Setup 1](assets/openwebui1.png)

- Select `Connections` form the left tab.
  - Enter URL that is matching this format: `http://YOUR_SERVER_PUBLIC_IP:PORT_NUMBER/v1`
  - Enter `API Key` matching the key you chose in step 3 and passed to `vllm serve`.
  - Enter model name that matches exactly the argument passed to `vllm serve` in step 3. For example `deepseek-ai/DeepSeek-R1-Distill-Qwen-32B`. 
  - Click on `+` button.
  - Click on `Save` button. 

![OpenWebUI Setup 1](assets/openwebui2.png)


<a id="step5"></a>

## Step 5: Chatbot Testing with DeepSeek-R1
Use Open WebUI to interact with your chatbot. Example prompt:

```
Imagine facing east. Turn 90° left, then 270° right, then 180° left. Which direction are you facing?
```

Follow up with a request for code visualization:

```
Can you give me a simple python code without importing external libraries to visualize this step-by-step with Unicode arrows?
```

![OpenWebUI Example](assets/webui_example.gif)

<a id="step6"></a>

## Step 6: Code Development Assistant Using CodeGPT or VS Code AI Toolkit
### For CodeGPT (This extentsion might not be functional in certain countries):
Install the CodeGPT extension in VS Code:
- Open VS Code
- Navigate to Extensions (`Ctrl+Shift+X`)
- Search and install **CodeGPT**
- Configure the extension with your custom endpoint URL to connect with your DeepSeek-R1 deployment.

Configure your endpoint URL in the Open WebUI client as follows:

- Create an account.
- Once logged in select view more as shown in the image below: 

![CodeGPT Setup 1](assets/codegpt_setup1.png)

- Select LLMs Cloud Model tab 
- Under `Select Provider` select `Custom`
- Under `Select model` enter model name that matches exactly the argument passed to `vllm serve` in step 3. For example `deepseek-ai/DeepSeek-R1-Distill-Qwen-32B`.
- Under `Connect to Custom`:
  - Enter `API Key` matching the key you chose in step 3 and passed to `vllm serve`.
  - Enter `Custom Link` matching this format: `http://YOUR_SERVER_PUBLIC_IP:PORT_NUMBER/v1/chat/completions`

![CodeGPT Setup 2](assets/codegpt_setup2.png)

### For VS Code AI Toolkit 
Install the AI Toolkit for VS Code extension in VS Code:
- Open VS Code
- Navigate to Extensions (`Ctrl+Shift+X`)
- Search and install **VS Code AI Toolkit**
- Click on `remote inference` as shown in the image below:

![AI Toolkit Setup 1](assets/aitoolkit1.png)

- Select `Add a custom model`

![AI Toolkit Setup 2](assets/aitoolkit2.png)

- Enter Open AI compatible URL matching this format: `http://YOUR_SERVER_PUBLIC_IP:PORT_NUMBER/v1/chat/completions`

![AI Toolkit Setup 3](assets/aitoolkit_url.png)

- Enter model name that matches exactly the argument passed to `vllm serve` in step 3. For example `deepseek-ai/DeepSeek-R1-Distill-Qwen-32B`.

![AI Toolkit Setup 4](assets/aitoolkit3.png)

- Press enter to display the model name as is. 

![AI Toolkit Setup 5](assets/aitoolkit5.png)

- Enter HTTP header for authorization matching this format `Authorization: Bearer API KEY` exactly as specified here where `API Key` must match the key you chose in step 3 and passed to `vllm serve`. If you use the exact same command provided in this tutorials enter this `Authorization: Bearer abc-123`.

![AI Toolkit Setup 6](assets/aitoolkit6.png)

Once you've completed the steps above, your model should appear listed under `My MODELS` on the left. Click your model to start the corresponding playground.

![AI Toolkit Setup 7](assets/aitoolkit.png)

<a id="step7"></a>

## Step 7: Build a Snake Game with CodeGPT
In VS Code, request:

```
"Can you build a classic snake game? Include 'Powered by DeepSeek-R1 on AMD MI300X' in the corner. Use Python."
```

Run the generated code and enjoy your game!

![CodeGPT Generated Snake Game](assets/snake.gif)

<a id="step8"></a>

## Step 8: Optional Advanced Challenge - Pac-Man
Try building a Pac-Man game with a maximum of three prompts. Here's an example built with a single prompt (DeepSeekR1 671B):

![CodeGPT Generated Pacman with Single Prompt](assets/pacman.gif)

<a id="step9"></a>

## Step 9: Deploying Your Web-Based AI Agent
Clone our customized fork from this [GitHub Repository](https://github.com/Mahdi-CV/web-ui-sglang-vllm-deepseekr-fork).

Once you start the agent on your web browser, follow the steps below to connect it to our vLLM server:
- Select the `Agent Settings` tab and make sure to uncheck `Use Vision` since this model does not accept images as input.

![Agent Setup 1](assets/agent1.png)

- Select `LLM Configuration` tab.
  - Select `openai` as your `LLM Provider`.
  - Type model name that matches exactly the argument passed to `vllm serve` in step 3. For example `deepseek-ai/DeepSeek-R1-Distill-Qwen-32B`.
  - Enter URL that is matching this format: `http://YOUR_SERVER_PUBLIC_IP:PORT_NUMBER/v1`.
  - Enter `API Key` matching the key you chose in step 3 and passed to `vllm serve`.

![Agent Setup 2](assets/agent2.png)

- Select the `Browser Settings` tab and enable Headless Mode`.

![Agent Setup 3](assets/agent3.png)

- Select `Run Agent` tab and give the agent a task to complete.

![Agent Setup 4](assets/agent4.png)


Follow the README instructions to install, configure, and launch the agent. Explore the examples below:

**Example 1:** Finding the most popular trending Hugging Face text-to-image model and generating a sample image:

![Web Agent Hugging Face Example](assets/hf_webagent.gif)

**Example 2:** Online shopping by gathering recipe ingredients and placing them in my shopping cart:

![Web Agent Dinner Recipe Shopping Example](assets/dinner_webagent.gif)

---

Happy coding! If you encounter issues or have questions, don’t hesitate to ask during the workshop!
